# LOCC Irreversibility - Data Analysis

**Reproducing results from:**  
*Experimental Demonstration of Entanglement Irreversibility*

This notebook walks through the complete analysis of hardware data.

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import json
from scipy import stats

# Plot settings
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

## 1. Load Hardware Data

In [ ]:
# Load data
with open('../data/hardware_results_20260129_113343.json', 'r') as f:
    data = json.load(f)

# Extract arrays
theta = np.array(data['data']['theta'])
C_standard = np.array(data['data']['C_standard'])
C_no_reversal = np.array(data['data']['C_no_reversal'])
C_with_reversal = np.array(data['data']['C_with_reversal'])
errors = np.array(data['data']['C_standard_err'])

print("Data loaded successfully!")
print(f"Coupling angles: {theta}°")
print(f"Number of shots: {data['metadata']['n_shots']}")

## 2. Key Result at θ = 90°

In [ ]:
# Find θ = 90° index
idx_90 = np.where(theta == 90)[0][0]

C_std = C_standard[idx_90]
C_rev = C_with_reversal[idx_90]
err = errors[idx_90]

print("Results at θ = 90° (maximum coupling):")
print("=" * 50)
print(f"Standard Bell:     C = {C_std:+.3f} ± {err:.3f}")
print(f"With reversal:     C = {C_rev:+.3f} ± {err:.3f}")
print(f"\nRestoration gap:   ΔC = {abs(C_std - C_rev):.3f}")

# Significance
gap = abs(C_std - C_rev)
combined_err = np.sqrt(2 * err**2)
significance = gap / combined_err

print(f"Statistical significance: {significance:.1f}σ")
print(f"\n{'✓ CONFIRMED' if significance > 5 else '✗ NOT CONFIRMED'}: Restoration FAILS")

## 3. Visualize Main Result

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

conditions = ['Standard\nBell', 'No\nReversal', 'With\nReversal']
values = [C_standard[idx_90], C_no_reversal[idx_90], C_with_reversal[idx_90]]
colors = ['#1f77b4', '#d62728', '#ff7f0e']

bars = ax.bar(conditions, values, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax.errorbar(range(3), values, yerr=[err]*3, fmt='none', color='black', capsize=5, linewidth=2)

ax.axhline(0, color='gray', linestyle='--', alpha=0.5)
ax.set_ylabel('Correlation C', fontsize=12, fontweight='bold')
ax.set_title('Complete Correlation Destruction (θ = 90°)', fontsize=14, fontweight='bold')
ax.set_ylim(-0.2, 1.0)

# Add significance annotation
ax.text(1, 0.9, f'Gap: {gap:.2f}\n({significance:.0f}σ)', 
        ha='center', fontsize=10, 
        bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))

plt.tight_layout()
plt.show()

## 4. Angular Dependence

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(theta))
width = 0.25

ax.bar(x - width, C_standard, width, label='Standard Bell',
       color='#1f77b4', alpha=0.7, yerr=errors, capsize=3)
ax.bar(x, C_no_reversal, width, label='No Reversal',
       color='#d62728', alpha=0.7, yerr=errors, capsize=3)
ax.bar(x + width, C_with_reversal, width, label='With Reversal',
       color='#ff7f0e', alpha=0.7, yerr=errors, capsize=3)

ax.axhline(0, color='black', linestyle='-', alpha=0.3, linewidth=1)
ax.set_xlabel('Coupling Angle θ (degrees)', fontsize=12, fontweight='bold')
ax.set_ylabel('Correlation C', fontsize=12, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels([f'{int(t)}°' for t in theta])
ax.legend(fontsize=11, frameon=True, shadow=True)
ax.set_title('Correlation vs Coupling Angle', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Statistical Tests

In [ ]:
print("Statistical Analysis")
print("=" * 60)

# Test 1: Restoration failure
print("\nTest 1: Does reversal restore correlation?")
print("-" * 60)
gap = abs(C_standard[idx_90] - C_with_reversal[idx_90])
sigma = gap / np.sqrt(2 * errors[idx_90]**2)
p_value = 2 * (1 - stats.norm.cdf(sigma))

print(f"Gap: ΔC = {gap:.3f}")
print(f"Significance: {sigma:.1f}σ")
print(f"p-value: {p_value:.2e}")
print(f"Result: {'FAILS' if sigma > 5 else 'succeeds'} to restore")

# Test 2: Erasure vs no erasure
print("\nTest 2: Does erasure affect correlation?")
print("-" * 60)
diff = abs(C_no_reversal[idx_90] - C_with_reversal[idx_90])
sigma_diff = diff / np.sqrt(2 * errors[idx_90]**2)

print(f"Difference: {diff:.3f}")
print(f"Significance: {sigma_diff:.1f}σ")
print(f"Result: {'Same' if sigma_diff < 2 else 'Different'}")
print(f"\nConclusion: Erasure has {'NO' if sigma_diff < 2 else 'some'} effect on correlation")

## 6. Hardware Quality

In [ ]:
print("Hardware Performance")
print("=" * 60)

mean_bell = np.mean(C_standard)
std_bell = np.std(C_standard)

print(f"\nBell state correlation:")
print(f"  Mean: {mean_bell:.3f}")
print(f"  Std:  {std_bell:.3f}")
print(f"\nGate fidelity: ~{mean_bell*100:.1f}%")
print(f"Expected ideal: 100%")
print(f"\nHardware quality: {'Excellent' if mean_bell > 0.80 else 'Good' if mean_bell > 0.70 else 'Fair'}")

## Summary

**Key findings:**
1. Standard Bell state shows C ≈ 0.84 (hardware limited)
2. Controlled rotation destroys correlation (C → 0)
3. Quantum erasure FAILS to restore correlation (C remains ≈ 0)
4. Gap is 0.89 with 28.5σ significance

**Conclusion:**  
Local operations cannot increase entanglement (LOCC impossibility confirmed experimentally)